# LSTM multiklasszifikációs problémára

## Feladat rövid leírása
A feladat során a uci-news-aggregator-small.csv felhasználásával mutatom be az LSTM alkalmazását. 
- Az eredeti, nagyobb méretű adathalmaznál a Test set eredménye 3 epoch esetén:
  - Loss: 0.299
  - Accuracy: 0.896
- A kisméretű fájlnál a Test set eredménye 10 epoch esetén:
  - Loss: 0.382
  - Accuracy: 0.873
  
Az eredeti adathalmaz megtalálható:
https://www.kaggle.com/datasets/uciml/news-aggregator-dataset

## Adatok előkészítése

### Importok

In [ ]:
import numpy as np 
import pandas as pd 
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.models import Sequential
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

### Nyers adat beolvasása
Az adathalmazból a title és category oszlopokat használom fel.
- TITLE : Az adott cikknek a címe.
- KATEGÓRIA : a hír kategóriáját írja le.
    - b: üzlet
    - t: tudomány és technika
    - e: szórakozás
    - m: egészség

In [ ]:
data = pd.read_csv('../data/uci-news-aggregator-small.csv', usecols=['TITLE', 'CATEGORY'])

##### Megnézzük, hogy az adott szövegben a kategóriák kiegyensúlyoztottak-e

In [ ]:
data.CATEGORY.value_counts()

#### Adatok kiegyensúlyozottá tétele

In [ ]:
num_of_categories = 45000
shuffled = data.reindex(np.random.permutation(data.index))
e = shuffled[shuffled['CATEGORY'] == 'e'][:num_of_categories]
b = shuffled[shuffled['CATEGORY'] == 'b'][:num_of_categories]
t = shuffled[shuffled['CATEGORY'] == 't'][:num_of_categories]
m = shuffled[shuffled['CATEGORY'] == 'm'][:num_of_categories]
concated = pd.concat([e,b,t,m], ignore_index=True)
#Shuffle the dataset
concated = concated.reindex(np.random.permutation(concated.index)) #get the indexes of a DataFrame as a list
concated['LABEL'] = 0

In [ ]:
concated.CATEGORY.value_counts()

### One hot-encoding

- Kategorikus jellemzők feldolgozására használjuk. 
- Ez a típusú kódolás új bináris jellemzőt hoz létre minden lehetséges kategóriához,  
és 1-es értéket rendel minden minta azon jellemzőjéhez, amely megfelel az eredeti kategóriájának.
- N darab elem esetén a k-adik elem one hot reprezentációja:[ 0 0 0 ... 1 ... 0]
- A névleges/szöveges kategoriális adatokat számértékekkel rendelkező jellemzőkké alakítja.

| Jellemző |                     
|  :---:   |
| Alma     | 
| Körte    |
| Körte    | 
| Alma     |


<div align="center">Encode után:</div>

| Alma    | Körte | 
| :---:   | :----:| 
| 1       | 0     |
| 0       | 1     | 
| 0       | 1     | 
| 1       |   0   | 
   

In [ ]:
concated.loc[concated['CATEGORY'] == 'e', 'LABEL'] = 0
concated.loc[concated['CATEGORY'] == 'b', 'LABEL'] = 1
concated.loc[concated['CATEGORY'] == 't', 'LABEL'] = 2
concated.loc[concated['CATEGORY'] == 'm', 'LABEL'] = 3
print(concated['LABEL'][:10])
labels = to_categorical(concated['LABEL'], num_classes=4)
print(labels[:10])
if 'CATEGORY' in concated.keys():
    concated.drop(['CATEGORY'], axis=1)

### Adatok előfeldolgozása
Kiszedjük a szövegből a filterben felsorolt nem elfogadott karaktereket, és a lowerrel kisbetűssé tesszük.  
- fit_on_texts: 
  - Szógyakoriság alapján hozza létre a szótár indexeit.
  - szó->index szótár, minden szó egyedi indexet kap.
  - Pl: "The cat sat on the mat." A szótára: word_index["the"] = 1; word_index["cat"] = 2 ...
- texts_to_sequences:
  - Minden egyes szót átvesz a szövegben, és lecseréli a megfelelő egész értékre a word_index szótárból.
- word_index: 
  - megadja az elkészített vektor hosszát.

In [ ]:
max_features = 4000
max_len = 100
tokenizer = Tokenizer(num_words=max_features, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(concated['TITLE'].values)
sequences = tokenizer.texts_to_sequences(concated['TITLE'].values)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X = pad_sequences(sequences, maxlen=max_len)

### Az adatok train és teszt adathalmazra bontása

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X , labels, test_size=0.25, random_state=42)

### A modell megvalósítása

In [ ]:
epochs = 10 
emb_dim = 32
batch_size = 32
labels[:2]

In [ ]:
model = Sequential()
model.add(Embedding(max_features, emb_dim, input_length=max_len))
model.add(SpatialDropout1D(0.7))
model.add(LSTM(64, dropout=0.7, recurrent_dropout=0.7))
model.add(Dense(4, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

print(model.summary())
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2,
                    callbacks=[EarlyStopping(monitor='val_loss',patience=2)])

#### Az elért eredmény pontossága

In [ ]:
accr = model.evaluate(X_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

### A tanulási és validációs pontosság, veszteség ábrázolása 

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'g', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.ylabel('value')
plt.xlabel('No. epoch')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'g', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.ylabel('value')
plt.xlabel('No. epoch')
plt.legend()

plt.show()

### A modell tesztelése txt bemenetekre
Megpróbáljuk a modellel megjósolni, hogy az adott mondatok melyik kategóriába fognak tartozni: 
- üzlet 
- tudomány 
- szórakozás
- egészség

Az 1. szöveg fordítása: a cukros üdítők súlygyarapodáshoz vezetnek.   
A várt kategória:health.        

In [ ]:
txt = ["Sugar-sweetened beverages are associated with weight gain."]
seq = tokenizer.texts_to_sequences(txt)
padded = pad_sequences(seq, maxlen=max_len)
pred = model.predict(padded)
labels = ['entertainment', 'bussiness', 'science/tech', 'health']
print(pred, labels[np.argmax(pred)])

A 2. szöveg fordítása: A COD játékkal játszani szórakoztató.   
A várt kategória:entertaintment.    

In [ ]:
txt = ["Playing with COD is fun."]
seq = tokenizer.texts_to_sequences(txt)
padded = pad_sequences(seq, maxlen=max_len)
pred = model.predict(padded)
labels = ['entertainment', 'bussiness', 'science/tech', 'health']
print(pred, labels[np.argmax(pred)])

A 3. szöveg fordítása:Eladásra kerülhet a Manchester United, a világ egyik legnagyobb futballklubja.  
A várt kategória: business. 

In [ ]:
txt = ["Manchester United, is potentially up for sale."]
seq = tokenizer.texts_to_sequences(txt)
padded = pad_sequences(seq, maxlen=max_len)
pred = model.predict(padded)
labels = ['entertainment', 'bussiness', 'science/tech', 'health']
print(pred, labels[np.argmax(pred)])

A 4. szöveg fordítása:A Meta létrehozott egy mesterséges intelligencia-algoritmust, amely képes legyőzni az embereket a Diplomácia nevű társasjátékban.  
A várt kategória: science/tech. 

In [ ]:
txt = ["Meta created an AI algorithm that can beat beat humans in the board game Diplomacy."]
seq = tokenizer.texts_to_sequences(txt)
padded = pad_sequences(seq, maxlen=max_len)
pred = model.predict(padded)
labels = ['entertainment', 'bussiness', 'science/tech', 'health']
print(pred, labels[np.argmax(pred)])